# Checking RIVM data dor corralations

In [1]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import scipy.stats
from scipy.stats import chisquare
import seaborn as sns

In [2]:
%run ./data/data.ipynb

Pulling data from sources.json:
(Source: CBS)  deceased @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=70895ned&_theme=76


KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
rivm_per_day.drop(columns=['prev_low',
                   'prev_up',
                   'Hospital_admission_NICE_notification',
                   'IC_admission_notification',
                   'Rt_low',
                   'Rt_up'], inplace=True)
rivm_per_day.dropna(inplace=True)
rivm_per_day

In [ ]:
# check for normality: https://towardsdatascience.com/normality-tests-in-python-31e04aa4f411
for c in rivm_per_day.columns:
    print(c)
    stat, p = chisquare(rivm_per_day[c])
    print('stat=%.3f, p=%.3f' % (stat,p))
    if p > 0.05:
        print('Probably Gaussian')
    else:
        print('Probably not Gaussian')
    print()

In [ ]:
def corr_visual(df, method):
    matrix = df.corr(method=method)
    plt.figure(figsize=(20, 8))
    sns.heatmap(matrix.replace(1,np.nan), annot=True, cmap="RdYlGn", vmin=-1, vmax=1)

In [ ]:
corr_visual(rivm_per_day, 'kendall')

In [ ]:
corr_visual(rivm_per_day['2020-06-01':], 'kendall')

## regression

In [ ]:
def regress(x_label, y_label, x, y):
    slope, intercept, r, p, stderr = scipy.stats.linregress(x,y)
    line = f'Regression line: y={intercept:.2f}+{slope:.2f}x, r={r:.2f}'
    print(line)
    
    fig, ax = plt.subplots()
    ax.plot(x, y, linewidth=0, marker='s', label='Data points')
    ax.plot(x, intercept + slope * x, label=line)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.legend(facecolor='white')
    plt.show()

In [ ]:
c = 0
len_c = len(rivm.columns)
for column in rivm.columns:
    cn = 1 + c
    while cn < len_c:
        cc = rivm.columns[c]
        ccn = rivm.columns[cn]
        print(cc + ',' + ccn)
        x = rivm[cc]
        y = rivm[ccn]
        regress(cc, cn, x, y)
        cn+=1
    c+=1